In [127]:
import pandas as pd

In [117]:
df = pd.read_csv(r'..\data\raw\realtor-data-clean.csv')

In [118]:
df['bed_bath_ratio'] = df['bed'] / (df['bath'] + 1e-6)

In [119]:
df['price_per_sqft'] = df['price'] / df['house_size']

In [120]:
df['total_rooms'] = df['bed'] + df['bath']

In [121]:
price_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Luxury': 3}
df['price_category'] = df['price_category'].map(price_map)

In [122]:
property_map = {'House':0,'Mansion':1}
df['property_type'] = df['property_type'].map(property_map)

In [123]:
df.drop(columns=['brokered_by', 'street','city'], inplace=True)

In [124]:
df['bed_group'] = df['bed_group'].replace('10+', '11').astype(int)
df['bath_group'] = df['bath_group'].replace('10+', '11').astype(int)

In [125]:
df['bed_group_10o+'] = df['bed_group'].apply(lambda x: 1 if x == 11 else 0)
df['bath_group_10o+'] = df['bath_group'].apply(lambda x: 1 if x == 11 else 0)

In [126]:
df.drop(columns=['bed', 'bath'], inplace=True)

In [128]:
df = pd.get_dummies(df, columns=['state', 'status'], prefix=['state', 'status'],dtype=int)